In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import os
import pandas as pd
CURR_DIR = os.getcwd()
print(CURR_DIR)

C:\Users\lulol\Desktop\Maestria\SistemasRecomendacion\Talleres\sr_taller_2\preprocesamiento


In [3]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
from sklearn.pipeline import FeatureUnion
from scipy.sparse import coo_matrix
import simplejson as json
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from mlxtend.feature_selection import ColumnSelector
from transformers  import *
from collections import Counter

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


## Unir reviews con negocios de restaurantes

In [4]:
review_json_path = './yelp_dataset/yelp_academic_dataset_review.json'
i= 0
size= 1000000
review = pd.read_json(review_json_path, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)

In [173]:

print ('Cargar negocio-')
columns = ('business_id', 'name','categories','attributes','city','stars')
business_json_path = CURR_DIR+'/yelp_dataset/yelp_academic_dataset_business.json'

business = pd.read_json(business_json_path, lines=True).sort_values('business_id')

# Trimming the dataset by city
business_by_city = business['city'] == "Boulder"
business = business[business_by_city]

# Trimming the dataset by the category 'Restaurants'
business_frame = business
count = 0
business_filter = df_b[df_b['categories'].str.contains(
                        'Restaurants',    
    case=False, na=False)]
print (len(business_frame))
print ('Cargando sólo restaurantes en boulder')

Cargar negocio-
666
Cargando sólo restaurantes en boulder


In [174]:
# There are multiple chunks to be read
chunk_list = []
for chunk_review in review:
    # Drop columns that aren't needed
    chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    # Inner merge with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(business_filter, chunk_review, on='business_id', how='inner')
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

7787 out of 1,000,000 related reviews
8006 out of 1,000,000 related reviews
10315 out of 1,000,000 related reviews
8772 out of 1,000,000 related reviews
10858 out of 1,000,000 related reviews
8872 out of 1,000,000 related reviews
8214 out of 1,000,000 related reviews
6119 out of 1,000,000 related reviews
5080 out of 1,000,000 related reviews


In [177]:
df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,user_id,review_stars,text,date
0,-B5r632_Gv8ZgVCbHI1DcQ,Subway,4550 Broadway St,Boulder,CO,80304,40.057139,-105.281794,2.5,9,0,"{'Alcohol': 'u'none'', 'RestaurantsGoodForGrou...","Fast Food, Restaurants, Sandwiches","{'Monday': '7:30-20:30', 'Tuesday': '7:30-20:3...",oqbvDg2THwdw_zSRe45JxA,1,Honestly the customer service sux. There rude ...,2016-05-19 18:03:14
1,-B5r632_Gv8ZgVCbHI1DcQ,Subway,4550 Broadway St,Boulder,CO,80304,40.057139,-105.281794,2.5,9,0,"{'Alcohol': 'u'none'', 'RestaurantsGoodForGrou...","Fast Food, Restaurants, Sandwiches","{'Monday': '7:30-20:30', 'Tuesday': '7:30-20:3...",gHmRQYAmCWmnKX-g65JRFw,4,"It's a subway, the food was fresh and they mad...",2015-05-29 20:34:43
2,-YwMfMAoIt5a_UaRB0fIHA,The Pita Pit,1509 Arapahoe Ave,Boulder,CO,80302,40.014881,-105.274330,3.0,23,0,"{'Ambience': '{'romantic': False, 'intimate': ...","Greek, Restaurants, Mediterranean","{'Monday': '10:30-0:0', 'Tuesday': '10:30-0:0'...",w5XMx7iqLyym91AlkZ6VDg,1,Short.... Because this place isn't worth much....,2014-04-02 00:53:15
3,-YwMfMAoIt5a_UaRB0fIHA,The Pita Pit,1509 Arapahoe Ave,Boulder,CO,80302,40.014881,-105.274330,3.0,23,0,"{'Ambience': '{'romantic': False, 'intimate': ...","Greek, Restaurants, Mediterranean","{'Monday': '10:30-0:0', 'Tuesday': '10:30-0:0'...",HvSgNiSZ9ps_PeH2VFry-g,3,I fell in love with the pita pit when I was go...,2012-11-08 04:18:53
4,-YwMfMAoIt5a_UaRB0fIHA,The Pita Pit,1509 Arapahoe Ave,Boulder,CO,80302,40.014881,-105.274330,3.0,23,0,"{'Ambience': '{'romantic': False, 'intimate': ...","Greek, Restaurants, Mediterranean","{'Monday': '10:30-0:0', 'Tuesday': '10:30-0:0'...",DAuahL9MPeX__73O0WMnyw,4,I drove into Boulder and wanted something othe...,2011-03-22 00:17:49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74018,yZm5tTTyZaq8vxfcnQYu-A,Extreme Pita,"2525 Arapahoe Ave, Ste E-1b",Boulder,CO,80302,40.014635,-105.262742,2.5,12,0,"{'GoodForKids': 'True', 'Ambience': '{'romanti...","Restaurants, Sandwiches","{'Monday': '10:30-22:0', 'Tuesday': '10:30-22:...",flPPMB7ZjA1JCu3ME7iePg,1,I almost never write reviews about chain-resta...,2011-09-07 01:10:25
74019,yZm5tTTyZaq8vxfcnQYu-A,Extreme Pita,"2525 Arapahoe Ave, Ste E-1b",Boulder,CO,80302,40.014635,-105.262742,2.5,12,0,"{'GoodForKids': 'True', 'Ambience': '{'romanti...","Restaurants, Sandwiches","{'Monday': '10:30-22:0', 'Tuesday': '10:30-22:...",ecgOCqFrBVHlduHePJPDBw,2,"When I first walked into this place, I had to ...",2011-06-06 16:02:41
74020,yZm5tTTyZaq8vxfcnQYu-A,Extreme Pita,"2525 Arapahoe Ave, Ste E-1b",Boulder,CO,80302,40.014635,-105.262742,2.5,12,0,"{'GoodForKids': 'True', 'Ambience': '{'romanti...","Restaurants, Sandwiches","{'Monday': '10:30-22:0', 'Tuesday': '10:30-22:...",PJ19W2lFWYQGbq6hivfURQ,4,I thought it was a good experience. The two e...,2011-11-20 03:59:16
74021,yZm5tTTyZaq8vxfcnQYu-A,Extreme Pita,"2525 Arapahoe Ave, Ste E-1b",Boulder,CO,80302,40.014635,-105.262742,2.5,12,0,"{'GoodForKids': 'True', 'Ambience': '{'romanti...","Restaurants, Sandwiches","{'Monday': '10:30-22:0', 'Tuesday': '10:30-22:...",E7ImoF4v6_gdle1Mde_OeA,1,"This review is belated, but the memory is oh s...",2012-10-02 01:27:15


In [175]:
csv_name = "yelp_reviews_restaurantes.csv"
df.to_csv(csv_name, index=False)

## Cargar top restaurantes y lugares que están abiertos

In [8]:

print ('Cargar negocio-')
columns = ('business_id', 'name','categories','attributes','city','stars')
business_json_path = CURR_DIR+'/yelp_dataset/yelp_academic_dataset_business.json'

business = pd.read_json(business_json_path, lines=True).sort_values('business_id')
business = business[business['is_open']==1]

# Trimming the dataset by top cities


options = [
'Austin',
'Portland',
'Vancouver',
'Atlanta',
'Orlando',
'Boston',
'Columbus',
'Boulder',
'Beaverton',
'Cambridge',
'Kissimmee',
'Burnaby',
'Richmond',
'Decatur',
'Winter Park',
'Somerville',
'Quincy',
'North Vancouver',
'Tigard',
'Brookline']
    
# selecting rows based on condition 
rslt_df = business[business['city'].isin(options)] 
#business_by_city = business['city'] == "Boulder"
#business = business[business_by_city]

# Trimming the dataset by top 20 categoriesb
business_filter = rslt_df[rslt_df['categories'].str.contains(
                        'Restaurants|Food|Shopping|Home Services|Health & Medical| \
                        Beauty & Spas|Local Services|Automotive|Event Planning & Services| \
                        Nightlife|Active Life|Bars|Coffee & Tea|Hotels & Travel|Sandwiches| \
                        Hair Salons|Fashion|Real Estate|Home & Garden|Auto Repair',    
    case=False, na=False)]
print (len(business_filter))
print ('Cargando sólo restaurantes del top')

Cargar negocio-
73111
Cargando sólo restaurantes del top


In [9]:
business_filter.count()

business_id     73111
name            73111
address         73111
city            73111
state           73111
postal_code     73111
latitude        73111
longitude       73111
stars           73111
review_count    73111
is_open         73111
attributes      68021
categories      73111
hours           63135
dtype: int64

In [12]:
# There are multiple chunks to be read
chunk_list = []
i = 0
for chunk_review in review:
    # Drop columns that aren't needed
    chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    # Inner merge with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(business_filter, chunk_review, on='business_id', how='inner')
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)
    i=i+1
    if i >500000:
        break
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

578711 out of 1,000,000 related reviews
596986 out of 1,000,000 related reviews
587251 out of 1,000,000 related reviews
575712 out of 1,000,000 related reviews
581757 out of 1,000,000 related reviews
592388 out of 1,000,000 related reviews
570321 out of 1,000,000 related reviews
375179 out of 1,000,000 related reviews


In [16]:
df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,user_id,review_stars,text,date
0,--0r8K_AQ4FZfLsX3ZYRDA,Blades of Glory,2761 Kensington Pl E,Columbus,OH,43202,40.021555,-83.00928,5.0,5,1,"{'BusinessAcceptsBitcoin': 'False', 'BusinessA...","Knife Sharpening, Home Services, Local Service...","{'Monday': '7:0-21:0', 'Tuesday': '7:0-21:0', ...",HzbScQ9XNEUrr8B_cL8i9g,5,Quick and reasonably priced service. Dropped o...,2020-04-15 13:33:08
1,--0r8K_AQ4FZfLsX3ZYRDA,Blades of Glory,2761 Kensington Pl E,Columbus,OH,43202,40.021555,-83.00928,5.0,5,1,"{'BusinessAcceptsBitcoin': 'False', 'BusinessA...","Knife Sharpening, Home Services, Local Service...","{'Monday': '7:0-21:0', 'Tuesday': '7:0-21:0', ...",0OsSL00kvQvMoOxf9sCRwA,5,Just as promised. Quick service and reasonable...,2019-10-26 23:08:46
2,--0r8K_AQ4FZfLsX3ZYRDA,Blades of Glory,2761 Kensington Pl E,Columbus,OH,43202,40.021555,-83.00928,5.0,5,1,"{'BusinessAcceptsBitcoin': 'False', 'BusinessA...","Knife Sharpening, Home Services, Local Service...","{'Monday': '7:0-21:0', 'Tuesday': '7:0-21:0', ...",dY8qFJdscFiQgC7NtT37aw,5,I'm so glad I found this place!!! I've tried a...,2017-11-15 20:19:50
3,--0r8K_AQ4FZfLsX3ZYRDA,Blades of Glory,2761 Kensington Pl E,Columbus,OH,43202,40.021555,-83.00928,5.0,5,1,"{'BusinessAcceptsBitcoin': 'False', 'BusinessA...","Knife Sharpening, Home Services, Local Service...","{'Monday': '7:0-21:0', 'Tuesday': '7:0-21:0', ...",1ixRBXu2YUaFoKXFTwrvcw,5,Super fast and the knives are sharp. He was su...,2017-09-03 17:15:48
4,--0r8K_AQ4FZfLsX3ZYRDA,Blades of Glory,2761 Kensington Pl E,Columbus,OH,43202,40.021555,-83.00928,5.0,5,1,"{'BusinessAcceptsBitcoin': 'False', 'BusinessA...","Knife Sharpening, Home Services, Local Service...","{'Monday': '7:0-21:0', 'Tuesday': '7:0-21:0', ...",RB_J91Ur9DlbbHZlVXZLjA,5,I am really happy with this business. I loved...,2019-11-06 20:20:03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4458300,zzpmoTVq4yn86U7ArHyFBQ,T4,18365 NW West Union Rd,Portland,OR,97229,45.558409,-122.86578,4.0,31,1,"{'BusinessAcceptsCreditCards': 'True', 'Caters...","Juice Bars & Smoothies, Waffles, Bubble Tea, F...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:30',...",KaBmPjjDRtDcW3t8q3U9dw,4,"This is a totally typical bubble tea place, wi...",2020-08-15 22:03:13
4458301,zzpmoTVq4yn86U7ArHyFBQ,T4,18365 NW West Union Rd,Portland,OR,97229,45.558409,-122.86578,4.0,31,1,"{'BusinessAcceptsCreditCards': 'True', 'Caters...","Juice Bars & Smoothies, Waffles, Bubble Tea, F...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:30',...",z9B25fKtfrxzUws_OOHJSQ,4,My teen is obsessed with boba. So on a Sunday ...,2020-12-21 14:35:07
4458302,zzpmoTVq4yn86U7ArHyFBQ,T4,18365 NW West Union Rd,Portland,OR,97229,45.558409,-122.86578,4.0,31,1,"{'BusinessAcceptsCreditCards': 'True', 'Caters...","Juice Bars & Smoothies, Waffles, Bubble Tea, F...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:30',...",yxrUKSA7dIvnwfJFGYTpQg,5,Great place for a snack. The various teas and...,2020-09-08 01:48:44
4458303,zzpmoTVq4yn86U7ArHyFBQ,T4,18365 NW West Union Rd,Portland,OR,97229,45.558409,-122.86578,4.0,31,1,"{'BusinessAcceptsCreditCards': 'True', 'Caters...","Juice Bars & Smoothies, Waffles, Bubble Tea, F...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:30',...",3Pi1X9CeCvaS_xfd9tprDg,5,I just want to commend the excellent hiring ch...,2020-10-02 20:58:48


In [18]:
csv_name = "yelp_reviews_restaurantes_top20ciudadycategoria1000000.csv"
df.head(1000000).to_csv(csv_name, index=False)

In [19]:
csv_name = "yelp_reviews_restaurantes_top20ciudadycategoria.csv"
df.to_csv(csv_name, index=False)

## cargar reviews todos

In [181]:
# There are multiple chunks to be read
chunk_list = []
for chunk_review in review:
    # Drop columns that aren't needed
    chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    # Show feedback on progress
    print(f"{chunk_review.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_review)
# After trimming down the review file, concatenate all relevant data back to one dataframe
df_completo = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)
print ('cargar reviews')

1000000 out of 1,000,000 related reviews
1000000 out of 1,000,000 related reviews
1000000 out of 1,000,000 related reviews
1000000 out of 1,000,000 related reviews
1000000 out of 1,000,000 related reviews
1000000 out of 1,000,000 related reviews
1000000 out of 1,000,000 related reviews
635403 out of 1,000,000 related reviews
cargar reviews


In [183]:
user_counts = df_completo['user_id'].value_counts(ascending=False)

In [211]:
active_users = user_counts.loc[user_counts >= 5].index.tolist()
df_completo = df_completo.loc[df_completo.user_id.isin(active_users)]

In [210]:
df_completo.describe

<bound method NDFrame.describe of                         user_id             business_id  review_stars  \
1        WoccjO9wbW44dbLcYBvDGA  vGO77OJg_0gBa8CnIoDfww             4   
2        VGW91-8bPOcXG9Itdck7Uw  YKrKQ5opZHxHZj4ayn-2Mg             5   
3        rq5J47QpfgQSDynJZIbjlw  gJWbeL6cDgcPvvZSnxo6cQ             5   
4        GlSFeRCj6oxl-RZ3PkFspQ  LvjVeqMsF1iYXLcnxHP1aw             4   
5        RtGqdDBvvBCjcu5dUqwfzA  LwrkmCr8sluLuASKNVN-GQ             2   
...                         ...                     ...           ...   
7635388  1WGPvc_cDXt-IPVNqg5BOA  FxveeHL_B0Kkz1KjPKyF3A             5   
7635392  mCanGeC78QwvqwSLpnZM_Q  K2XwYhodpeGWW6T57bZ88Q             5   
7635393  spFkIHa7x15DJe4gVnSCqw  XVmp1LmrXKB8cXomLKbu9w             5   
7635400  tr13Jb83h2itjyXVwaO5eA  rbuj2X4SXIc3MDul4dcxIA             5   
7635402  HsT8bz3-SKMk7YidwLLM6Q  pxwYIa1jZzHJ4R0Sp1cHFw             5   

                                                      text  \
1        This was my first 

In [207]:
active_users=df_completo['user_id'].unique()
active_users

array(['WoccjO9wbW44dbLcYBvDGA', 'VGW91-8bPOcXG9Itdck7Uw',
       'rq5J47QpfgQSDynJZIbjlw', ..., '6Yx-7IxgsDR3X9LqxJN1sg',
       'FieSKJG4K0gD4kIB8f0dkw', 'zQx8eofz4u0J3cmc5C5P8g'], dtype=object)

In [213]:
csv_name = "active_user.csv"
pd.DataFrame(active_users).to_csv(csv_name, index=False)

## cargar usuarios

In [30]:
df_users= pd.DataFrame(columns = ['user_id', 'name','average_stars'])

CURR_DIR = os.getcwd()
print(CURR_DIR)

chunksize = 10 #** 6

user_path = CURR_DIR+'/yelp_dataset/yelp_academic_dataset_user.json'
#user_path = CURR_DIR+'/yelp_dataset/yelp_user_5.json'
with  pd.read_json(user_path, lines=True,chunksize=chunksize) as reader:
    for chunk in reader:
        print(f"{chunk.shape[0]} out of {chunksize:,} ")
        df_users = df_users.append(chunk[['user_id', 'name','average_stars']])
        if i==0:
            break
#print(df_users)
print ('cargar usuario')

C:\Users\lulol\Desktop\Maestria\SistemasRecomendacion\Talleres\sr_taller_2\preprocesamiento
10 out of 10 
cargar usuario


In [225]:
business_json_path = CURR_DIR+'/yelp_dataset/yelp_academic_dataset_business.json'
business_all = pd.read_json(business_json_path, lines=True).sort_values('business_id')
#df_explode = df.assign(categories = df.categories
                         .str.split(', ')).explode('categories')
#df_explode

In [227]:
#pd.DataFrame(Counter(df_explode.categories.values).most_common(20), columns=['categories','businesses'])

,latitude,longitude,stars,review_count,is_open
count,160585.000000,160585.000000,160585.000000,160585.000000,160585.000000
mean,38.759794,-94.266212,3.656954,51.964548,0.767494
std,7.138042,19.975446,0.943604,130.030448,0.422431
min,27.998972,-123.393929,1.000000,5.000000,0.000000
25%,30.355886,-122.589583,3.000000,8.000000,1.000000
50%,42.177366,-84.383281,4.000000,17.000000,1.000000
75%,45.458531,-81.288501,4.500000,44.000000,1.000000
max,49.490000,71.113271,5.000000,9185.000000,1.000000
